In [ ]:
from google.colab import drive
drive.mount('/mnt/drive')

Mounted at /mnt/drive


In [ ]:
import pandas as pd

!pip install spacy
!python -m spacy download de_core_news_sm

In [ ]:
df = pd.read_csv("/content/Schwurbelarchiv.csv")
df.head()

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")
/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,ID,From Name,Datetime,Message Text,Photo,Video Duration,Forwarded From Name,Forwarded Message Text,Forwarded Photo,Forwarded Video Duration
0,ff627252-b067-4ace-8baf-521c71313377,Oliver Janich Top News,2020-10-18 17:47:20,\nHier findet Ihr ab jetzt alle meine Videos u...,NaN,NaN,NaN,NaN,NaN,NaN
1,e4116e63-6c66-42f9-8419-0f53fbba8427,Oliver Janich Top News,2020-10-18 17:47:52,"\n▶️ <strong>Nena, Wendler, Ronaldos Schwester...",NaN,NaN,Oliver Janich öffentlich,"\n▶️ <strong>Nena, Wendler, Ronaldos Schwester...",NaN,NaN
2,a017f93b-7515-41cd-a699-1c7eee9a12de,Oliver Janich Top News,2020-10-18 19:07:30,\nBiden ist erledigt. Das wollte ich morgen ei...,NaN,NaN,Oliver Janich öffentlich,\nBiden ist erledigt. Das wollte ich morgen ei...,NaN,NaN
3,bff400b7-47a4-4e7a-b853-b9f420324f12,Oliver Janich Top News,2020-10-19 18:09:15,\n▶️ <strong>Schrecklicher Fund auf Biden-Lapt...,NaN,NaN,Oliver Janich öffentlich,\n▶️ <strong>Schrecklicher Fund auf Biden-Lapt...,NaN,NaN
4,90a4ce25-f7a7-4181-9c79-88fbccc1010c,Oliver Janich Top News,2020-10-20 07:34:19,"\n50Cent wählt Trump, weil die Steuern sonst i...",NaN,NaN,Oliver Janich öffentlich,"\n50Cent wählt Trump, weil die Steuern sonst i...",NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        187 non-null    object
 1   From Name                 187 non-null    object
 2   Datetime                  187 non-null    object
 3   Message Text              160 non-null    object
 4   Photo                     34 non-null     object
 5   Video Duration            19 non-null     object
 6   Forwarded From Name       180 non-null    object
 7   Forwarded Message Text    154 non-null    object
 8   Forwarded Photo           41 non-null     object
 9   Forwarded Video Duration  22 non-null     object
dtypes: object(10)
memory usage: 14.7+ KB


In [ ]:
len(df)

187

In [ ]:
group_names_list = df["name"].tolist()

In [ ]:
len(group_names_list)

187

In [ ]:
# Ersetzen von np.nan Werten durch einen leeren String
group_names = [name if type(name) == str else "" for name in group_names_list]


In [ ]:
group_names

['\nHier findet Ihr ab jetzt alle meine Videos und wenige Top-Beiträge von Kollegen. In der Regel keine Nachrichten.\n       ',
 '\n▶️ <strong>Nena, Wendler, Ronaldos Schwester, 10.000 Mediziner: Auf unserer Seite! CNN &amp; NYT gegen mich</strong><br/><br/>💬 Exklusive Inhalte, Frühzugriff und Kommentierung meiner Videos im Premium-Kanal<br/><br/><a href="https://youtu.be/3C4H-lfh3MA">https://youtu.be/3C4H-lfh3MA</a>\n',
 '\nBiden ist erledigt. Das wollte ich morgen eigentlich mit Jan Walter besprechen. Wusste nicht, dass Alex Jones schon alles hat. Interview mache ich trotzdem. <a href="https://censoredbyjack.com/watch?id=5f8b7cca9d7ca5144824dc5e">https://censoredbyjack.com/watch?id=5f8b7cca9d7ca5144824dc5e</a>\n',
 '\n▶️ <strong>Schrecklicher Fund auf Biden-Laptop: Warum handelt FBI nicht?</strong><br/><br/>💬 Exklusive Inhalte, Frühzugriff und Kommentierung meiner Videos im Premium-Kanal<br/><br/><a href="https://youtu.be/-J2o8xP7_kk">https://youtu.be/-J2o8xP7_kk</a>\n',
 '\n50Cent w

Erst Pre-Processing der Gruppennamen


In [ ]:
import spacy
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Deutsche Stopwörter von NLTK
stop_words_nltk = set(stopwords.words('german'))

# Initialisieren von spaCy für Deutsch
nlp = spacy.load('de_core_news_sm')

# Update spaCy's Stopwort-Liste mit NLTK Stopwörtern (optional)
nlp.Defaults.stop_words.update(stop_words_nltk)

# Funktion um zu überprüfen, ob ein Token ein Stopwort ist
def is_stopword(word):
    return word in nlp.Defaults.stop_words

def remove_emojis_and_special_char(text):
    emoji_and_special_char_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Symbole & Piktogramme
                           u"\U0001F680-\U0001F6FF"  # Transport & Karte
                           u"\U0001F1E0-\U0001F1FF"  # Flaggen
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"  # Verschiedene Symbole
                           r"\|®"  # Hinzufügen von | und ® zur Entfernung
                           "]+", flags=re.UNICODE)
    return emoji_and_special_char_pattern.sub(r'', text)  # Ersetzt Emojis durch leeren String

def preprocess(text):
    # Entfernen von Emojis
    text_no_emoji_and_special_char = remove_emojis_and_special_char(text)

    # Verarbeiten des Textes ohne Emojis mit spaCy
    doc = nlp(text_no_emoji_and_special_char)

    # Tokenisieren, Stopwörter entfernen und Lemmatisieren und Kleinschreibung auf alles anwenden
    tokens = [token.lemma_.lower() for token in doc if not is_stopword(token.text) and not token.is_punct]

    return tokens

# Anwenden der Vorverarbeitung
preprocessed_group_names = [preprocess(text) for text in group_names_list]
for tokens in preprocessed_group_names:
    print(tokens)


In [ ]:
# Zusammenführen der Token zurück zu Strings
preprocessed_group_names_as_str = [' '.join(tokens) for tokens in preprocessed_group_names]

In [ ]:
preprocessed_group_names_as_str

['der echteelternstehenauf sachsen-anhalt',
 'wolfmut tiefgang',
 'roh vegan limit',
 '  infokanal bewegung zwickau',
 'corona wahrheit instagram',
 'michael wendler',
 'diplomateninterview',
 'morgellons-nanotechnologie',
 'pegzda patriotisch europäer zionisierung abendland',
 'frag wwg1wga',
 'elternstehenauf pirna',
 'coronasaurus',
 'farben lieben leben',
 'real great reset  ',
 '  us wahl 2020 live ticker',
 'der wächter',
 'markus möller   feed',
 'elternstehenauf frankenthal_ludwigshafen worms bad dürkheim donnerbergkreis',
 'news infos hand hand kl',
 'afd-freunde tübingen',
 'vmzmvril neu-schwabenland',
 'kontakt nord',
 'plandemie',
 'kenfm ken jebsen fanchat diskussion',
 '  berlin stehen auf diskussion chat',
 'zwischen himmel erde',
 'freiheitsboten_augsburgzentrum',
 'interessant informativ plauderecke',
 'aktiver widerstand + aktion definition video foto mensch hier keine demoterminegrafik @demotermine',
 'wachsamewolfsburger patriot',
 'conspiracy facts pedogate',
 'cor

Versuch von Topic Modeling with Bertopic; die Daten sind aktuell eine Liste mit vielen Listen.

In [ ]:
pip install bertopic
from bertopic import BERTopic

Anzahl der min_topic_size ausprobieren:

*   bei min 25 Dokumente pro Topic gibt es über 2700 Outlier
*   bei min 20 Dokumente, sind es 2308 outlier
*   bei min 15 Dokumente, sind es immer noch 2034
*   Entscheidung für min_topic_size = 10





In [ ]:
docs = preprocessed_group_names_as_str

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model, min_topic_size=10)
topics = topic_model.fit_transform(docs)

In [ ]:
# Get an overview of the topics found by the model
topic_info = topic_model.get_topic_info()

topic_info.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1599,-1_stellensuche_stehen_stellenangebote_ffentlich,"[stellensuche, stehen, stellenangebote, ffentl...",[ stellenangebote stellensuche original für u...
1,0,99,0_wahrheit_wahrheits_wahrheitsfindung_wahrhaft...,"[wahrheit, wahrheits, wahrheitsfindung, wahrha...","[der wahrheit lügen, ⁉wahrheit lügen⁉, ⁉wahrhe..."
2,1,92,1_diskussion_diskussionskanal_diskussionsrunde...,"[diskussion, diskussionskanal, diskussionsrund...","[diskussion, bürgeraufbruch diskussion, real_h..."
3,2,89,2_eva_offiziell_herman_christina,"[eva, offiziell, herman, christina, nina, alex...","[eva herman offiziell, eva herman offiziell, e..."
4,3,88,3_videoaktivisten_videocesar_videoarchiv_video...,"[videoaktivisten, videocesar, videoarchiv, vid...","[datei bild video sichern, datei bild video si..."


In [ ]:
# Speichern der Topic-Informationen als CSV-Datei
topic_info.to_csv("topic_info_10doc_per_topic.csv", index=False)

In [ ]:
labels = topic_model.generate_topic_labels()

In [ ]:
labels

['-1_deutsch_chat_tages',
 '0_dassindwir_dassindwirwir_dassindwirgraubnden',
 '1_videoaktivisten_videocesar_videoarchiv',
 '2_coronabildersammlung_coronafaktendiskussion_coronaaufklaerung',
 '3_covide19_covid19_covid19impfstoff',
 '4_gesundheit_gesundheitswesen_gesundheitsratgeber',
 '5_impffrei_impffreiwork_impffreisiege',
 '6_freiheit_freiheitlich_aufklrung',
 '7_offiziell_wisnewski_inoffiziell',
 '8_diskussionsgruppe_diskussionsraum_diskussion',
 '9_mensch_menschsein_menschseinszentren',
 '10_gruppe_gruppenbersicht_gruppenregel',
 '11_patriot_patriotenheilbronn_patriotenkanal',
 '12_helfen_hand_ein',
 '13_wahrheitsbewegung_verfassungsfreunde_chat',
 '14_compacttv_blaupausetv_tv',
 '15_elternstehenauf_elternstehenaufcelle_elternstehenaufneumarktopf',
 '16_elternnetzwerk_elternnetzwerklrrach_elternnetz',
 '17_freidenker_freiberg_freigeist',
 '18_deutschland_deutschlandtreff_deutschlandkurier',
 '19_frankfurt_berlin_brandenburg',
 '20_strahlenkranz_polizeigewalt_privatinvestor',
 '21_g

In [ ]:
# Example: Get keywords for the first few topics
for topic_num in range(min(len(topic_info), 10)):  # Zeige nur die ersten 10 Topics, wenn verfügbar
    topic_keywords = topic_model.get_topic(topic_num)
    print(f"Topic {topic_num}:")
    print(topic_keywords)


Topic 0:
[('coronafakt', 0.80567855), ('corona', 0.79516196), ('coronaler', 0.7204591), ('coronaimpfung', 0.7088953), ('coronadiskussio', 0.70395267), ('coronademonstrationsankndigung', 0.6836077), ('coronash', 0.683424), ('coronawahn', 0.6655453), ('coronaausschussde', 0.6605077), ('coronakriseinfos', 0.6555622)]
Topic 1:
[('wahrheit', 0.7557794), ('wahrheits', 0.74421775), ('wahrheitsfindung', 0.6898782), ('wahrhaftigkeit', 0.6894908), ('wahrheitsbewegung', 0.6857735), ('wahrheitstrumer', 0.6814902), ('wahrheitenorg', 0.6702999), ('wahrheitfriedenaufklrung', 0.6365589), ('wahrheitskanal', 0.619572), ('wahlbetrug', 0.49083346)]
Topic 2:
[('chat', 0.6070998), ('chatungeimpft', 0.60336226), ('chatquerdenken40osten', 0.57992566), ('chatroom', 0.5606425), ('chatquerdenken40westen', 0.54413176), ('chatgruppe', 0.54375815), ('wahrheitsbewegung', 0.5350843), ('chatraum', 0.51865005), ('verfassungsbeschwerde', 0.50601906), ('verfassungsfreunde', 0.5004977)]
Topic 3:
[('dassindwir', 0.8686223)

In [ ]:
# Visualize the topics
topic_model.visualize_barchart(top_n_topics=10)


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1272,-1_deutsch_chat_tages_markus,"[deutsch, chat, tages, markus, querdenke, hain...","[deutsch tages news dtn chat, sara bennett lig..."
1,0,82,0_dassindwir_dassindwirwir_dassindwirgraubnden...,"[dassindwir, dassindwirwir, dassindwirgraubnde...","[dassindwir oberösterreich, dassindwir bawü, d..."
2,1,77,1_videoaktivisten_videocesar_videoarchiv_video...,"[videoaktivisten, videocesar, videoarchiv, vid...","[datei bild video sichern, datei bild video si..."
3,2,74,2_coronabildersammlung_coronafaktendiskussion_...,"[coronabildersammlung, coronafaktendiskussion,...","[corona-anzeig österreich, corona fehlalarm di..."
4,3,63,3_covide19_covid19_covid19impfstoff_covid19imp...,"[covide19, covid19, covid19impfstoff, covid19i...","[covid 19 impfpass, covid-19 impfung, impfpass..."
...,...,...,...,...,...
348,347,5,347_faschismus_fahne_deutschland_frei,"[faschismus, fahne, deutschland, frei, franz, ...",[stoppt brd faschismus hans franz friends ersa...
349,348,5,348_wahrhaftigkeit_nichts_wahrheit_2020,"[wahrhaftigkeit, nichts, wahrheit, 2020, , , ,...","[nichts wahrheit, nichts wahrheit, nichts wahr..."
350,349,5,349_ketzer_neuzeit_datenschtzer_untersttzer,"[ketzer, neuzeit, datenschtzer, untersttzer, s...","[bw löwen-elter neuzeit, ketzer neuzeit, ket..."
351,350,5,350_hinterfrager_hinter_lgen_der,"[hinterfrager, hinter, lgen, der, hilfe, stgt,...","[hinter kulisse nwo, stgt wd hinter lügen, stg..."


In [ ]:
fig = topic_model.visualize_barchart(top_n_topics=10)


In [ ]:
# Schriftgrößen anpassen
fig.update_layout(
    title_font_size=24,       # Größe des Diagrammtitels
    font_size=10,             # Größe der allgemeinen Schrift
    width=1300,   # Breite des Diagramms in Pixel
    height=800    # Höhe des Diagramms in Pixel
)


# TF-IDF:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

# Laden der deutschen Stopwörter
nltk.download('stopwords')
german_stop_words = stopwords.words('german')

# TF-IDF Vectorizer initialisieren
tfidf_vectorizer = TfidfVectorizer(token_pattern=r'\b[a-zA-Z]+\b',stop_words=german_stop_words)

# Zusammenführen der Token zurück zu Strings
preprocessed_group_names_as_str = [' '.join(tokens) for tokens in preprocessed_group_names]

# Gruppennamen transformieren
X_tfidf = tfidf_vectorizer.fit_transform(preprocessed_group_names_as_str)

# Schlüsselwörter basierend auf TF-IDF extrahieren
keywords_tfidf = tfidf_vectorizer.get_feature_names_out()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
keywords_tfidf

array(['a', 'aach', 'aachen', ..., 'zweifel', 'zweiter', 'zwickau'],
      dtype=object)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialisieren des TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(token_pattern=r'\b[a-zA-Z]+\b', stop_words = german_stop_words)

# Anwenden des Vectorizers auf Ihre Gruppennamen
X_tfidf = tfidf_vectorizer.fit_transform(preprocessed_group_names_as_str)

# Abrufen der Feature-Namen, die den Wörtern im Korpus entsprechen
feature_names = tfidf_vectorizer.get_feature_names_out()

# Abrufen der TF-IDF Matrix als Array
tfidf_matrix = X_tfidf.toarray()

# Anzeigen der TF-IDF Werte für jedes Wort in jedem Dokument
for doc_index, doc in enumerate(tfidf_matrix):
    print(f"Dokument {doc_index}:")
    for word_index, tfidf_value in enumerate(doc):
        if tfidf_value > 0:
            print(f"  {feature_names[word_index]}: {tfidf_value}")


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Dokument 5413:
  diebasis: 0.5336323613969034
  he: 0.65125044658198
  of: 0.5395455112376863
Dokument 5414:
  berlin: 0.3507396017786033
  diebasis: 0.4340588140927482
  hellersdorf: 0.5867600351304543
  marzahn: 0.5867600351304543
Dokument 5415:
  diebasis: 0.6178639117815135
  heidelberg: 0.7862850542379947
Dokument 5416:
  kinderschutz: 1.0
Dokument 5417:
  don: 0.459887711254855
  dttm: 0.48127783789267464
  media: 0.3572321098682913
  t: 0.35980601270085205
  the: 0.3194241062109449
  trust: 0.4447111748818844
Dokument 5418:
  oha: 0.8621276457602154
  patriot: 0.5066911509153762
Dokument 5419:
  mindofheart: 1.0
Dokument 5420:
  geesthacht: 0.8621276457602154
  patriot: 0.5066911509153762
Dokument 5421:
  dortmund: 0.826436573000462
  patriot: 0.563029831187702
Dokument 5422:
Dokument 5423:
  diebasis: 0.4369064808306935
  henning: 0.5906095066328105
  s: 0.3887877004359061
  verteiler: 0.5560011345967265
Dokumen

# Top N-Wörter extrahieren

In [ ]:
import numpy as np

def top_n_words_per_document(tfidf_matrix, feature_names, N=10):
    for doc_index, doc in enumerate(tfidf_matrix):
        top_indices = np.argsort(doc)[::-1][:N]
        top_features = [(feature_names[i], doc[i]) for i in top_indices if doc[i] > 0]
        print(f"Dokument {doc_index}:")
        for feature, value in top_features:
            print(f"  {feature}: {value}")

# Anwendung der Funktion
top_n_words_per_document(tfidf_matrix, feature_names, N=5)


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  immer: 0.8430644388075165
  frei: 0.5378125621609888
Dokument 5353:
  saarland: 0.5831948902479008
  ungeimpft: 0.508335582925519
  original: 0.4936410648373161
  deutschland: 0.3972368993771757
Dokument 5354:
  nordrhein: 0.5303865874775576
  westfalen: 0.5303865874775576
  ungeimpft: 0.41385321311343193
  original: 0.4018899083001876
  deutschland: 0.32340401242096145
Dokument 5355:
  school: 0.6542951741718822
  of: 0.4676313697827732
  ungeimpft: 0.4263664964490749
  original: 0.41404146864319613
Dokument 5356:
  hessen: 0.5727662054364202
  ungeimpft: 0.5129578584011799
  original: 0.49812972363756186
  deutschland: 0.4008489588900109
Dokument 5357:
  watching: 0.6737188023032938
  are: 0.5249208959066783
  we: 0.5201548120161397
Dokument 5358:
  schleswig: 0.5106690606828828
  holstein: 0.5064889865647351
  ungeimpft: 0.43475958227586115
  original: 0.42219193452433984
  deutschland: 0.33974121473822433
Dokument

In [ ]:
def top_n_words_in_dataset(tfidf_matrix, feature_names, N=20):
    # Summe der TF-IDF-Werte für jedes Wort über alle Dokumente berechnen
    word_tfidf_sum = np.sum(tfidf_matrix, axis=0)

    # Indizes der Top-N-Wörter erhalten
    top_n_indices = np.argsort(word_tfidf_sum)[-N:][::-1]

    # Die entsprechenden Wörter und ihre summierten TF-IDF-Werte auswählen
    top_n_words = feature_names[top_n_indices]
    top_n_values = word_tfidf_sum[top_n_indices]

    # Ausgabe der Top-N-Wörter mit ihren TF-IDF-Werten
    print(f"Top-{N} Wörter im gesamten Datensatz:")
    for word, value in zip(top_n_words, top_n_values):
        print(f"{word}: {value}")



In [ ]:
# Aufrufen der Funktion mit der TF-IDF-Matrix, den Feature-Namen und der gewünschten Anzahl von Top-Wörtern
top_n_words_in_dataset(tfidf_matrix, feature_names, N=20)

Top-20 Wörter im gesamten Datensatz:
chat: 182.3700824859167
kanal: 92.68206547993066
corona: 89.65080011430047
info: 80.6009948321181
freiheit: 77.69562547657733
deutschland: 74.4704425523354
stehen: 68.25611885319783
frei: 66.2728255711075
eltern: 55.512412744073345
diskussion: 54.07402213591228
berlin: 52.51123782398281
news: 51.66888844604621
gruppe: 51.431179163840376
q: 48.8170206802009
patriot: 47.585252013655136
wahrheit: 47.45592098458171
querdenken: 40.05943844676497
tv: 39.52695358025353
dassindwir: 37.49621995885203
elternnetzwerk: 36.55329293269367
